In [10]:
from bs4 import BeautifulSoup
import csv
import numpy as np
import pandas as pd
import requests
import re
import time
from tqdm import tqdm
import json
import pprint

In [ ]:
with open("../data/鉄道人身事故データベース.html", "r") as f:
    html = f.read()
    soup = BeautifulSoup(html, 'html.parser')
    a = soup.findAll("table")[0].findAll("tr")
    with open("../data/jinshin.csv", "w", newline="", encoding="utf-8") as file:
        csv_write = csv.writer(file)
        for tr in a:
            data = []
            for cell in tr.findAll(["td", "th"]):
                if (not "詳細" in cell.get_text()) and cell.get_text() != "":
                    data.append(cell.get_text())
            if len(data) == 3:
                data.append("")
            for url in tr.findAll("a"):
                data.append(url.get("href"))
            csv_write.writerow(data)

In [ ]:
df = pd.read_csv("../data/jinshin.csv", header=0, names=["date", "line", "place", "injury", "url"])
data = []
sentences = []
for url in tqdm(df["url"]):
    d = BeautifulSoup(requests.get(url).text,'html.parser')
    a = d.findAll("a",text=re.compile("魚拓"))
    if a :
        data.append(a[0].get("href"))
    else :
        data.append("")
    sentence = d.findAll("p",class_="card-text")
    if sentence:
        sentences.append(sentence[0].get_text().replace(" ","").replace("\n","").replace("　",""))
    else :
        sentences.append("")
    time.sleep(0.01)
# 保存
save_data = np.array(data)
save_sentences = np.array(sentences)
np.save("archive_url3",save_data)
np.save("news_sentences", save_sentences)
# 付け加えて保存
df["archive"] = data
df["sentence"] = sentences
df.to_csv("../data/jinshin_archive.csv")

In [ ]:
url = df["url"][0]
d = BeautifulSoup(requests.get(url).text, 'html.parser')

gyotaku = d.findAll("a", text =re.compile("魚拓"))[0].get("href")
print(gyotaku)
print(d.findAll('p',class_="card-text")[0].get_text())

In [9]:
df = pd.read_csv("../data/jinshin_archive.csv", header=0, names=["date", "line", "place", "injury", "url", "archive", "sentence"])
data = []
sentences = []
for url in tqdm(df["url"]):
    d = BeautifulSoup(requests.get(url).text,'html.parser')
    sentence = d.findAll("p",class_="card-text")
    if sentence:
        sentences.append(sentence[0].get_text().replace(" ","").replace("\n","").replace("　",""))
    else :
        sentences.append("")
    time.sleep(0.01)
# 保存
save_sentences = np.array(sentences)
np.save("news_sentences", save_sentences)
# 付け加えて保存
df["sentence"] = sentences
df.to_csv("../data/jinshin_archive_sentence.csv")

100%|██████████| 14897/14897 [1:01:16<00:00,  4.05it/s]


In [29]:
with open("../data/JR東/東日本旅客鉄道東海道線.geojson") as f:
    df = json.load(f)
print(df["features"][0]["properties"]["name"])

東京


In [35]:
df = pd.read_csv("../data/jinshin_archive_sentence.csv",index_col=0)
hoge = df.place=="東京駅"
df[hoge]

,date,line,place,injury,url,archive,sentence
1180,2021/08/16 09:02,横須賀線,東京駅,NaN,https://jinshinjiko.com/accidents/13714,NaN,NaN
2359,2020/06/15 17:01,京葉線,東京駅,NaN,https://jinshinjiko.com/accidents/12538,NaN,NaN
3877,2018/12/23 09:08,京浜東北線,東京駅,NaN,https://jinshinjiko.com/accidents/11019,NaN,NaN
5241,2017/09/24 13:53,山手線,東京駅,NaN,https://jinshinjiko.com/accidents/9655,NaN,NaN
5295,2017/09/06 22:08,京葉線,東京駅,NaN,https://jinshinjiko.com/accidents/9600,NaN,NaN
5728,2017/04/21 11:04,宇都宮線,東京駅,男性,https://jinshinjiko.com/accidents/9165,https://archive.is/e3IvV,２１日午前１１時５分ごろ、東京都千代田区の東京駅で、常磐線土浦発品川行きの特別快速電車（１５...
7394,2015/11/17 14:32,山手線,東京駅,男性,https://jinshinjiko.com/accidents/781,https://archive.is/goDXc,１７日午後２時半ごろ、東京都千代田区の東京駅構内の山手線ホームから人が落下する人身事故が発生...
8657,2014/10/18 15:52,京浜東北線,東京駅,30代/男性/死亡,https://jinshinjiko.com/accidents/1417,http://archive.is/phlKX,30代とみられる男性が大宮行きの列車にはねられ死亡。
9154,2014/05/09 12:28,京浜東北線,東京駅,NaN,https://jinshinjiko.com/accidents/1872,NaN,NaN
10126,2013/08/06 20:40,総武快速線,東京駅,NaN,https://jinshinjiko.com/accidents/2854,NaN,NaN
